In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io 
import scipy.misc 
import matplotlib.cm as cm 
import random 
import scipy.optimize 
import itertools
from scipy.special import expit 

In [6]:
data = scipy.io.loadmat( 'data/ex4data1.mat' )
y = data['y']
X = np.insert(data['X'],0,1,axis=1)

print("y shape: ", y.shape)
print("x shape: ", X.shape)

y shape:  (5000, 1)
x shape:  (5000, 401)


In [5]:
mat = scipy.io.loadmat( 'data/ex4weights.mat' )
Theta1, Theta2 = mat['Theta1'], mat['Theta2']

print("Theta1 shape: ", Theta1.shape)
print("Theta2 shape: ", Theta2.shape)

Theta1 shape: (25, 401)
Theta2 shape: (10, 26)


In [6]:
#Neural Netword Architecture:
input_layer_size = 400
hidden_layer_size = 25
output_layer_size = 10 
n_training_samples = X.shape[0]

In [7]:
def flatten_theta(thetas_list):
    flattened_list = [ mytheta.flatten() for mytheta in thetas_list ]
    combined = list(itertools.chain.from_iterable(flattened_list))
    assert len(combined) == (input_layer_size+1)*hidden_layer_size + \
                            (hidden_layer_size+1)*output_layer_size
    return np.array(combined).reshape((len(combined),1))

def reshape_theta(flattened_array):
    theta1 = flattened_array[:(input_layer_size+1)*hidden_layer_size] \
            .reshape((hidden_layer_size,input_layer_size+1))
    theta2 = flattened_array[(input_layer_size+1)*hidden_layer_size:] \
            .reshape((output_layer_size,hidden_layer_size+1))
    
    return [ theta1, theta2 ]

def flattenX(myX):
    return np.array(myX.flatten()).reshape((n_training_samples*(input_layer_size+1),1))

def reshapeX(flattenedX):
    return np.array(flattenedX).reshape((n_training_samples,input_layer_size+1))

In [8]:
def cost(mythetas_flattened,myX_flattened,myy,mylambda=0.):
    mythetas = reshape_theta(mythetas_flattened
    myX = reshapeX(myX_flattened)
    total_cost = 0.
    m = n_training_samples

    for irow in range(m):
        myrow = myX[irow]
        myhs = forward_propagate(myrow,mythetas)[-1][1]
        tmpy  = np.zeros((10,1))
        tmpy[myy[irow]-1] = 1
        mycost = -tmpy.T.dot(np.log(myhs))-(1-tmpy.T).dot(np.log(1-myhs))
        total_cost += mycost
  
    total_cost = float(total_cost) / m
    total_reg = 0.
    for mytheta in mythetas:
        total_reg += np.sum(mytheta*mytheta) 
    total_reg *= float(mylambda)/(2*m)
    return total_cost + total_reg    

def forward_propagate(row,Thetas):
    features = row
    zs_as_per_layer = []
    for i in range(len(Thetas)):  
        Theta = Thetas[i]
        z = Theta.dot(features).reshape((Theta.shape[0],1))
        a = expit(z)
        zs_as_per_layer.append( (z, a) )
        if i == len(Thetas)-1:
            return np.array(zs_as_per_layer)
        a = np.insert(a,0,1)
        features = a

In [9]:
myThetas = [ Theta1, Theta2 ]
print(cost(flatten_theta(myThetas),flattenX(X),y))

0.28762916516131876


#### Regularization

In [10]:
myThetas = [ Theta1, Theta2 ]
print(cost(flatten_theta(myThetas),flattenX(X),y,mylambda=1.))

0.3844877962428938


### 2 Backpropagation

#### 2.1 Sigmoid gradient

In [11]:
def sigmoid(z):
    dummy = expit(z)
    return dummy*(1-dummy)

#### 2.2 Random initialization

In [12]:
def random_theta():
    epsilon_init = 0.12
    theta1_shape = (hidden_layer_size, input_layer_size+1)
    theta2_shape = (output_layer_size, hidden_layer_size+1)
    rand_thetas = [ np.random.rand( *theta1_shape ) * 2 * epsilon_init - epsilon_init, \
                    np.random.rand( *theta2_shape ) * 2 * epsilon_init - epsilon_init]
    return rand_thetas

#### 2.3 Backpropagation


In [13]:
def backward_propagate(mythetas_flattened,myX_flattened,myy,mylambda=0.):
    mythetas = reshape_theta(mythetas_flattened)
    myX = reshapeX(myX_flattened)
    Delta1 = np.zeros((hidden_layer_size,input_layer_size+1))
    Delta2 = np.zeros((output_layer_size,hidden_layer_size+1))
    m = n_training_samples
    for irow in range(m):
        myrow = myX[irow]
        a1 = myrow.reshape((input_layer_size+1,1))
        temp = forward_propagate(myrow,mythetas)
        z2 = temp[0][0]
        a2 = temp[0][1]
        z3 = temp[1][0]
        a3 = temp[1][1]
        tmpy = np.zeros((10,1))
        tmpy[myy[irow]-1] = 1
        delta3 = a3 - tmpy 
        delta2 = mythetas[1].T[1:,:].dot(delta3)*sigmoid(z2) 
        a2 = np.insert(a2,0,1,axis=0)
        Delta1 += delta2.dot(a1.T) 
        Delta2 += delta3.dot(a2.T) 
        
    D1 = Delta1/float(m)
    D2 = Delta2/float(m)
    
    D1[:,1:] = D1[:,1:] + (float(mylambda)/m)*mythetas[0][:,1:]
    D2[:,1:] = D2[:,1:] + (float(mylambda)/m)*mythetas[1][:,1:]
    
    return flatten_theta([D1, D2]).flatten()

In [14]:
flattenedD1D2 = backward_propagate(flatten_theta(myThetas),flattenX(X),y,mylambda=0.)
D1, D2 = reshape_theta(flattenedD1D2)

#### 2.4 Gradient checking

In [15]:
def check_gradient(mythetas,myDs,myX,myy,mylambda=0.):
    myeps = 0.0001
    flattened = flatten_theta(mythetas)
    flattenedDs = flatten_theta(myDs)
    myX_flattened = flattenX(myX)
    n_elems = len(flattened) 
    for i in range(3):
        x = int(np.random.rand()*n_elems)
        epsvec = np.zeros((n_elems,1))
        epsvec[x] = myeps
        cost_high = cost(flattened + epsvec,myX_flattened,myy,mylambda)
        cost_low  = cost(flattened - epsvec,myX_flattened,myy,mylambda)
        mygrad = (cost_high - cost_low) / float(2*myeps)
        print("Element: %d. Numerical Gradient = %f. BackProp Gradient = %f."%(x,mygrad,flattenedDs[x]))

In [16]:
check_gradient(myThetas,[D1, D2],X,y)

Element: 6167. Numerical Gradient = -0.000014. BackProp Gradient = -0.000014.
Element: 5604. Numerical Gradient = -0.000006. BackProp Gradient = -0.000006.
Element: 121. Numerical Gradient = 0.000001. BackProp Gradient = 0.000001.


#### Learning  the Network

In [18]:
def train(mylambda=0.):
    randomThetas_unrolled = flatten_theta(random_theta())
    result = scipy.optimize.fmin_cg(cost, x0=randomThetas_unrolled, fprime=backward_propagate, \
                               args=(flattenX(X),y,mylambda),maxiter=50,disp=True,full_output=True)
    return reshape_theta(result[0])

In [1]:
learned_Thetas = train()

NameError: name 'trainNN' is not defined

In [ ]:
def predict(row,Thetas):
    classes = range(1,10) + [10]
    output = forward_propagate(row,Thetas)
    return classes[np.argmax(output[-1][1])] 

def accuracy(myX,myThetas,myy):
    n_correct, n_total = 0, myX.shape[0]
    for irow in xrange(n_total):
        if int(predict(myX[irow],myThetas)) == int(myy[irow]): 
            n_correct += 1
    print "Training set accuracy: %0.1f%%"%(100*(float(n_correct)/n_total))

In [ ]:
accuracy(X,learned_Thetas,y)

In [ ]:
learned_regularized_Thetas = train(mylambda=1.)

In [ ]:
accuracy(X,learned_regularized_Thetas,y)